In [39]:
import requests as reqs
import pandas as pd
import io
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, interact

# Govt website for data pull.
source = 'https://data.london.gov.uk/download/average-house-prices/b1b0079e-698c-4c0b-b8c7-aa6189590ca4/land-registry-house-prices-borough.csv'
source_get = reqs.get(source).content
# Capture data for handling - io used to not require local instance.
data_set = pd.read_csv(io.StringIO(source_get.decode('utf-8')), thousands=',')
data_set['Year'] = data_set['Year'].str[-4:].astype(int)
data_set['Value'] = data_set['Value'].astype(int)

In [40]:
# UI bits
borough_box=Dropdown(
        options=[x for x in data_set['Area'].unique()],
        value='London',
        description='Pick Area:',
        layout={'width': 'max-content'},
        style = {'description_width': 'initial'},
        disabled=False)
data_box=Dropdown(
        options=['Value'],
        value='Value',
        description='Plot Value:',
        layout={'width': 'max-content'},
        style = {'description_width': 'initial'},
        disabled=False)
year_box=Dropdown(
        options=[x for x in data_set['Year'].unique()],
        value=2010,
        description='Min Year:',
        layout={'width': 'max-content'},
        style = {'description_width': 'initial'},
        disabled=False)

In [41]:
@interact(borough = borough_box, pull_val = data_box, min_year = year_box)
def init_filter(borough, pull_val, min_year):
    data_adj = data_set.loc[data_set['Measure'] == 'Median']
    data_adj = data_adj.loc[data_adj['Area'] == borough]
    data_adj = data_adj.loc[data_adj['Year'] >= min_year]
    data_final = data_adj.groupby(['Year'])['Value'].mean()
    return data_final.plot.line(x='Year',y=pull_val)

interactive(children=(Dropdown(description='Pick Area:', index=39, layout=Layout(width='max-content'), options…